<a href="https://colab.research.google.com/github/BFvandy/Nissan-Text-Mining-POC-with-Transformer/blob/main/Reddit_Q_A_prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
cur_path = "/content/drive/My Drive/Nissan Internship/Reddit Comments"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/Nissan Internship/Reddit Comments




# Load Transformer


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.3 MB/s 
     |████████████████████████████████| 596 kB 34.3 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline

In [ ]:
model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

https://huggingface.co/deepset/roberta-base-squad2

In [ ]:
res

{'answer': 'gives freedom to the user',
 'end': 84,
 'score': 0.2117149531841278,
 'start': 59}

# Load Dataset & Adujst structure

In [ ]:
comments = pd.read_csv('reddit_all_preprocessed.csv', encoding_errors='ignore')
comments

,Make,Model,Source,Text,timestamp
0,Nissan,Rogue,Reddit,This is incredible timing for me as my cvt jus...,1.659024e+09
1,Nissan,Rogue,Reddit,Op do you own it out right? i'd sell it or tra...,1.659036e+09
2,Nissan,Rogue,Reddit,I have 2019 sport transmission failed at 42k m...,1.659033e+09
3,Nissan,Rogue,Reddit,Not yet. but i bought an extended warranty any...,1.659030e+09
4,Nissan,Rogue,Reddit,My 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09
...,...,...,...,...,...
28376,Toyota,RAV4,Reddit,"I would maybe wait for the fall, just so your ...",1.625451e+09
28377,Toyota,RAV4,Reddit,Yeah i probably will just wait until september...,1.625464e+09
28378,Toyota,RAV4,Reddit,Change it.,1.637008e+09
28379,Toyota,RAV4,Reddit,Did plenty of drain and fills on higher mileag...,1.637016e+09


In [ ]:
fun_labels = ['Dependable', 'Lasts long', 'Value for money', 'Quality fit and finish', 'Attractive styling'
                                    , 'Safe', 'Retains resale value', 'Driver comfort', 'Fun to drive', 
                                    'Advanced features', 'Responsive handling', 'Prestigious', 'Dealerships', 
        'Fuel efficient', 'Quick acceleration', 'Environmentally friendly', 'Affordable']
pers_labels = ['Trusted', 'Leader', 'Responsible', 'Confident', 'Innovative', 'Exciting', 'Practical', 'Adventurous', 
               'Passionate', 'Distinctive', 'Youthful', 'Aggressive']

all_labels = fun_labels+pers_labels
all_labels_lower = [i.lower() for i in all_labels]

In [ ]:
QA_input = {
    'question': 'Is value for money mentioned?',
    'context': comments.at[0, 'Text']
}
res = nlp(QA_input)
res

{'answer': 'incredible timing',
 'end': 25,
 'score': 2.6897398129221983e-05,
 'start': 8}

In [ ]:
comments[all_labels_lower] = pd.NaT
comments

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Rogue,Reddit,This is incredible timing for me as my cvt jus...,1.659024e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Nissan,Rogue,Reddit,Op do you own it out right? i'd sell it or tra...,1.659036e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Nissan,Rogue,Reddit,I have 2019 sport transmission failed at 42k m...,1.659033e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Nissan,Rogue,Reddit,Not yet. but i bought an extended warranty any...,1.659030e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Nissan,Rogue,Reddit,My 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28376,Toyota,RAV4,Reddit,"I would maybe wait for the fall, just so your ...",1.625451e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
28377,Toyota,RAV4,Reddit,Yeah i probably will just wait until september...,1.625464e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
28378,Toyota,RAV4,Reddit,Change it.,1.637008e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
28379,Toyota,RAV4,Reddit,Did plenty of drain and fills on higher mileag...,1.637016e+09,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


# Apply QA Model for prefiltering

In [ ]:
import time

start = time.time()

for i in range(len(comments)):
  for j in all_labels_lower:
    try:
      QA_input = {
          'question': 'Is ' + j + ' mentioned?',
          'context': comments.at[i, 'Text']
      }
      res = nlp(QA_input)
      comments.at[i, j] = 1 if res['score'] > 0.1 else 0 
    except:
      pass
  if i % 1000 == 0:
    print(i)
  else:
    pass
    
end = time.time()
print((end - start)/60)


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
110.54946519136429


In [ ]:
comments

,Make,Model,Source,Text,timestamp,dependable,lasts long,value for money,quality fit and finish,attractive styling,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Rogue,Reddit,This is incredible timing for me as my cvt jus...,1.659024e+09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nissan,Rogue,Reddit,Op do you own it out right? i'd sell it or tra...,1.659036e+09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nissan,Rogue,Reddit,I have 2019 sport transmission failed at 42k m...,1.659033e+09,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Nissan,Rogue,Reddit,Not yet. but i bought an extended warranty any...,1.659030e+09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Nissan,Rogue,Reddit,My 2016 maxima cvt took a shit at 85k miles lu...,1.659037e+09,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28376,Toyota,RAV4,Reddit,"I would maybe wait for the fall, just so your ...",1.625451e+09,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28377,Toyota,RAV4,Reddit,Yeah i probably will just wait until september...,1.625464e+09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28378,Toyota,RAV4,Reddit,Change it.,1.637008e+09,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28379,Toyota,RAV4,Reddit,Did plenty of drain and fills on higher mileag...,1.637016e+09,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
comments.to_csv('reddit_cmt_prefiltered.csv', index=False)
print("Save it")

Save it


In [ ]:
for i in range(len(all_labels_lower)):
  label = all_labels_lower[i]
  count = (comments[label] == 1).sum()
  print(label)
  print(count)

dependable
1481
lasts long
1806
value for money
934
quality fit and finish
1404
attractive styling
791
safe
769
retains resale value
510
driver comfort
1013
fun to drive
480
advanced features
1272
responsive handling
992
prestigious
1177
dealerships
1186
fuel efficient
937
quick acceleration
410
environmentally friendly
466
affordable
839
trusted
882
leader
401
responsible
2379
confident
997
innovative
962
exciting
851
practical
1750
adventurous
948
passionate
732
distinctive
1892
youthful
747
aggressive
1176
